# Desk sensor data to occupancy

This notebook is used to convert sensor data to desk occupancy data. 

the datadump contains information on all kinds of sensors, in this notebook we only consider the desk sensors and their occupancy state. 

The sensor data contains the timestamps on which the state of occupancy changes. For analysis purposes, I want to have the occupancy for a timeframe of 15 minutes instead of the instantaneous occupancy at a certain timestamp. 

## Assumptions

the desk sensor data is csv files with one file per day. We assume all occupancy states to reset to unoccupied at midnight. The .csv files have the following format:

| time | deviceName | fCnt | application | msgSource | gatewayMac | occupied | temC | humidity | co2 | sound | lux |

We filter to include entries where application contains the string "desk". Matching on cases could be more performant, but I'm unsure what application names are used. 

*time* is the message timestamp in posixct format. 
*deviceName* is the name of the desk sensor.
*fCnt* is a rolling counter for the messages sent by the sensor, which resets on reconnect. Missing values indicate dropped/lost messages
*application* is the application name of the sensor. We filter to include only desk sensors.
*occupied* is the occupancy state of the desk sensor, with 1 indicating occupied and 0 indicating unoccupied.

Other columns are not relevant for this analysis.

All sensors send out a 'heartbeat' message every ~12 hours. This message contains the last know occupancy state, which should result in consecutive messages with the same occupancy state. We will filter these out.

Unoccupancy is defined as no activity for 30 minutes. Therefore we will subtract 30 minutes from the timestamp of unoccupancy. 


# Aims

Calculating the overall occupancy over all desk sensors per 15-minute intervals. If occupation does not span the entire interval, the ratio occupied in the interval is used. 


The sensor_export_chris directory contains two folders which should not be considered: "archive" and "captainExport_NL_UTC_UTRECHT". All other folders in the sensor_export_directory contain the sensor logs per year. The individual files per day are named "export_assetData-NL-UTC-UTRECHTLROF_YYYYMMDD.csv"

In [ ]:
# cell 1: imports & settings
import os
from glob import glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil import parser

# parameters
DELAY_MINUTES = 30
EXTRA_RELEASE_MINUTES = 15 + DELAY_MINUTES
INPUT_DIR = "sensor_export_chris/NL-UTC-UTRECHTLROF_2025"
OUTPUT_DIR = "processed_data"
ACTIVE_SENSORS_FILE = "sensor_export_chris/captainExport_NL-UTC-UTRECHT/donecaptainExport_NL-UTC-UTRECHT_202404270525.csv"

# cell 2: load active sensors
active_sensors_df = pd.read_csv(ACTIVE_SENSORS_FILE, na_values=[""], sep=";")
active_sensors = set(active_sensors_df["sensor"].dropna().astype(str))

# cell 3: core functions
def data_to_occupation_events(sensor_filepath):
    # read & filter
    sensor_filepath = ""
    df = pd.read_csv(sensor_filepath, na_values=[""], sep=";")    
    print(df.head())

    df = df[df["deviceName"].isin(active_sensors)]
    if df.empty:
        return pd.DataFrame()    # nothing to do

    # select & clean
    df = df[["time", "deviceName", "occupied", "application"]]
    df = df[df["application"].isin([
        "officesense_desk",
        "officesense_desk_EU868",
        "officesense_desk_EU868_gk"
    ])].drop(columns="application")
    df["occupied"] = df["occupied"].replace(2, 1)
    df = df[df["occupied"].isin([0,1])]

    # parse timestamps
    df["time"] = pd.to_datetime(df["time"], format="%d-%m-%Y %H:%M:%S", errors="coerce")
    df = df.dropna(subset=["time"])
    df = df.sort_values(["deviceName","time"])

    # remove heartbeat duplicates
    df["prev_occ"] = df.groupby("deviceName")["occupied"].shift()
    df = df[df["occupied"] != df["prev_occ"]].drop(columns="prev_occ")

    # drop initial zeros per device
    def drop_initial_zero(g):
        if len(g)>0 and g["occupied"].iat[0]==0:
            return g.iloc[1:]
        return g
    df = df.groupby("deviceName", group_keys=False).apply(drop_initial_zero)

    # append a forced “unoccupied” if last state is 1
    last = df.groupby("deviceName", group_keys=False).tail(1)
    last = last[last["occupied"]==1]
    if not last.empty:
        last = last.copy()
        last["time"] = last["time"] + timedelta(minutes=EXTRA_RELEASE_MINUTES)
        last["occupied"] = 0
        df = pd.concat([df, last], ignore_index=True)

    # subtract delay from every 0‐event
    df.loc[df["occupied"]==0, "time"] = df.loc[df["occupied"]==0, "time"] - timedelta(minutes=DELAY_MINUTES)
    df = df.sort_values(["deviceName","time"])

    # build start/end events
    events = []
    for device, g in df.groupby("deviceName"):
        starts = g.loc[g["occupied"]==1, "time"].values
        ends   = g.loc[g["occupied"]==0, "time"].values
        n = min(len(starts), len(ends))
        if n>0:
            for i in range(n):
                duration = (ends[i] - starts[i]).total_seconds()/60.0
                if duration >= 1:
                    events.append({
                        "deviceName": device,
                        "start": starts[i],
                        "end": ends[i],
                        "duration": round(duration,2)
                    })
    return pd.DataFrame(events)

def calculate_active_intervals(events_df):
    if events_df.empty:
        return pd.DataFrame(columns=["interval","active_sensors"])

    # floor to 15min bins
    events_df["start"] = pd.to_datetime(events_df["start"])
    events_df["end"]   = pd.to_datetime(events_df["end"])
    events_df["start"] = events_df["start"].dt.floor("15T")
    events_df["end"]   = events_df["end"].dt.floor("15T")

    # expand each event into 15‑min timesteps
    rows = []
    for _, row in events_df.iterrows():
        spans = pd.date_range(row.start, row.end, freq="15T")
        for t in spans:
            rows.append((row.deviceName, t))
    long = pd.DataFrame(rows, columns=["deviceName","interval"])
    long = long.drop_duplicates()

    # count active sensors per interval
    counts = (long
              .groupby("interval")["deviceName"]
              .nunique()
              .reset_index(name="active_sensors"))

    # fill in missing intervals
    full = pd.DataFrame({
        "interval": pd.date_range(counts.interval.min(),
                                  counts.interval.max(),
                                  freq="15T")
    })
    full = full.merge(counts, on="interval", how="left").fillna(0)
    full["active_sensors"] = full["active_sensors"].astype(int)
    return full

def process_file(fn):
    events = data_to_occupation_events(os.path.join(INPUT_DIR, fn))
    if events.empty:
        # no output
        return None
    timeline = calculate_active_intervals(events)
    outpath = os.path.join(OUTPUT_DIR, fn)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    timeline.to_csv(outpath, sep=";", decimal=",", index=False)
    return fn

# cell 4: batch‐process files sequentially
all_input = [os.path.basename(x) for x in glob(os.path.join(INPUT_DIR,"*.csv"))]
all_output = set(os.listdir(OUTPUT_DIR)) if os.path.isdir(OUTPUT_DIR) else set()
to_do = [f for f in all_input if f not in all_output][:10]

print(f"Processing {len(to_do)} files…")
for fn in to_do:
    res = process_file(fn)
    if res:
        print("✔", res)
    else:
        print("⏭ skipped")


Processing 10 files…


C:\Users\STBROUWE\AppData\Local\Temp\ipykernel_10960\443174530.py:23: DtypeWarning: Columns (2,6,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(sensor_filepath, na_values=[""], sep=";")


KeyError: 'deviceName'